In [1]:
import numpy as np

In [2]:
npoints=1000
dim=3

In [3]:
x1=np.random.random((npoints,dim))
x2=np.random.random((npoints,dim))
x3=np.random.random((npoints,dim))

In [4]:
x12=((x1-x2)**2).sum(axis=1)
x13=((x1-x3)**2).sum(axis=1)
x23=((x2-x3)**2).sum(axis=1)

In [5]:
ndelta=5000
D = np.random.random((ndelta))

In [6]:
def two_point(x12,D):
    return x12**D

In [7]:
def ICFT(x1,x2):
    result = np.zeros((npoints,dim,dim))
    def CD(i,j):
        if (i==j):
            return 1
        else:
            return 0
    
    for i in range(npoints):
        for j in range(dim):
            for k in range (dim):
                result[i,j,k]=CD(j,k)- (2*(x1[i,j]-x2[i,j])*(x1[i,k]-x2[i,k]))/x12[i]
    result = np.reshape(result,(npoints,dim*dim))
    return result

In [8]:
def IS(x1,x2):
    result = np.zeros((npoints,dim,dim))
    def CD(i,j):
        if (i==j):
            return 1
        else:
            return 0
    alpha = np.random.uniform(-3,-1)
    for i in range(npoints):
        for j in range(dim):
            for k in range (dim):
                result[i,j,k]=CD(j,k)+ (alpha*(x1[i,j]-x2[i,j])*(x1[i,k]-x2[i,k]))/x12[i]
    result = np.reshape(result,(npoints,dim*dim))
    return result

In [9]:
def f1f2CFT(x1,x2,D):
    for i in range(npoints):        
        d=D[i]*np.ones(dim*dim)
    result = ICFT(x1,x2)/d
    return result

In [10]:
def f1f2S(x1,x2,D):
    for i in range(npoints):        
        d=D[i]*np.ones(dim*dim)
    result = IS(x1,x2)/d
    return result

In [11]:
def cft_three_point(x1,x2,x3,D):
    Numerator=np.zeros((npoints,dim))
    
    x12=((x1-x2)**2).sum(axis=1)
    x13=((x1-x3)**2).sum(axis=1)
    x23=((x2-x3)**2).sum(axis=1)
    Denominator = (x12**D)*(x23**D)*(x13**D)
    
    for j in range(npoints):
        for i in range(dim):
            x1x3i = (x1[j][i]-x3[j][i])
            x2x3i = (x2[j][i]-x3[j][i])
            Numerator[j][i] = ((x1x3i*x23[j])-(x2x3i*x13[j]))/(np.sqrt(x12[j])*np.sqrt(x23[j])*np.sqrt(x13[j]))
            Numerator[j][i] = Numerator[j][i]/Denominator[j]
    return Numerator

In [12]:
def scale_three_point(x1,x2,x3,D):
    
    
    a=np.random.random()
    b=np.random.random()
    c=np.random.random()
    
    x12=((x1-x2)**2).sum(axis=1)
    x13=((x1-x3)**2).sum(axis=1)
    x23=((x2-x3)**2).sum(axis=1)
    
    
    tot=a+b+c
    dtot=3*D
    a=a*dtot/tot
    b=b*dtot/tot
    c=c*dtot/tot
    Denominator = (x12**a)*(x23**b)*(x13**c)
    Numerator=np.zeros((npoints,dim))
    
    for j in range(npoints):
        for i in range(dim):
            x1x3i = (x1[j][i]-x3[j][i])
            x2x3i = (x2[j][i]-x3[j][i])
            Numerator[j][i] = ((x1x3i*x23[j])-(x2x3i*x13[j]))/(np.sqrt(x12[j])*np.sqrt(x23[j])*np.sqrt(x13[j]))
            Numerator[j][i] = Numerator[j][i]/Denominator[j]
    return Numerator 
    
    
    

In [14]:
X=[]
Y=[]



for i in range(ndelta):
    
    d=D[i]*np.ones((npoints))
    d1=two_point(x12,d)
    

    cft=cft_three_point(x1,x2,x3,d)
    scale=scale_three_point(x1,x2,x3,d)
    
    phicft=f1f2CFT(x1,x2,d)
    phicft=np.array(phicft)
    phicft=np.reshape(phicft,(9,npoints))
    
    phis=f1f2S(x1,x2,d)
    phis=np.array(phis)
    phis=np.reshape(phis,(9,npoints))
    
    cft=np.hsplit(cft,dim)
    scale=np.hsplit(scale,dim)
    
    X.append([x12,x23,x13,d1,phicft[0,:],phicft[1,:],phicft[2,:],phicft[3,:],phicft[4,:],phicft[5,:],phicft[6,:],phicft[7,:],phicft[8,:],np.reshape(cft[0],npoints),np.reshape(cft[1],npoints),np.reshape(cft[2],npoints)])   
    Y.append([0])  
    
    X.append([x12,x23,x13,d1,phis[0,:],phis[1,:],phis[2,:],phis[3,:],phis[4,:],phis[5,:],phis[6,:],phis[7,:],phis[8,:],np.reshape(scale[0],npoints),np.reshape(scale[1],npoints),np.reshape(scale[2],npoints)])   
    Y.append([1])
    
    del(d1,cft,scale,phicft,phis)

In [15]:
X=np.array(X)
Y=np.array(Y)

In [18]:
X = X.reshape(X.shape[0], ((4+dim*(dim+1))*npoints))
data = np.concatenate((X,Y),axis=1)

In [20]:
import pickle
pickle.dump(data, open(r'C:\Users\Aditya\Desktop\ML Project\Training Data\Spin_Correlator.bin', "wb"))

In [21]:
del(data)